# Google Trends

In [1]:
import os
os.chdir("../../")

import inspect
import numpy as np
import pandas as pd
from scripts.python.GoogleTrends import GT

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Select, HoverTool
from bokeh.plotting import figure, show, output_notebook

output_notebook()

Loading BokehJS ...

The following code was modified from [Development Data Partership](https://datapartnership.org/). See [example code](https://github.com/datapartnership/docs-google/blob/master/examples/google-health-trends-api.ipynb).

In [2]:
print(inspect.getsource(GT))

class GT:
    def __init__(self, _GOOGLE_API_KEY = GoogleAPIkey):
        self.service = build(
            serviceName=SERVICE_NAME,
            version=SERVICE_VERSION,
            discoveryServiceUrl=_DISCOVERY_SERVICE_URL,
            developerKey=_GOOGLE_API_KEY,
            cache_discovery=False)
        self.block_until = None

    def get_health_trends(self, terms, timelineResolution="month"):
        graph = self.service.getTimelinesForHealth(
            terms=terms,
            timelineResolution=timelineResolution
        )

        try:
            response = graph.execute()
            return response

        except HttpError as http_error:
            data = json.loads(http_error.content.decode('utf-8'))
            code = data['error']['code']
            reason = data['error']['errors'][0]['reason']
            if code == 403 and reason == 'dailyLimitExceeded':
                self.block_until = datetime.combine(
                    date.today() + timedelta(days=1), d

In [3]:
folder = os.getcwd() + "/data/tourism/trends/"
filepaths = [folder + path for path in os.listdir(folder)]

In [4]:
trends = pd.DataFrame()
for file in filepaths:
    if "readme" not in file:
        country_trend = (pd.read_csv(file).drop("Unnamed: 0", axis=1)
                           .iloc[:, [0, -3, -2, -1]])
        country_trend.columns = [col.replace(" ", "_").lower()
                                 for col in country_trend.columns]
        country_trend["date"] = pd.to_datetime(country_trend["date"])

        if trends.empty:
            trends = country_trend
        else:
            trends = trends.merge(country_trend, how="left", on="date")

display(trends.head(5))

,date,palau_flights,palau_hotel,palau_travel,solomon_islands_flights,solomon_islands_hotel,solomon_islands_travel,tonga_flights,tonga_hotel,tonga_travel,vanuatu_flights,vanuatu_hotel,vanuatu_travel,fiji_flights,fiji_hotel,fiji_travel,samoa_flights,samoa_hotel,samoa_travel
0,2004-01-01,0.0,3.155717,3.055298,0.0,0.0,2.490457,0.000000,0.000000,0.000000,2.232933,3.102047,2.453441,8.480770,14.971975,25.644490,2.932001,6.204171,2.053351
1,2004-02-01,0.0,5.111154,2.142547,0.0,0.0,0.000000,2.256111,3.711238,0.000000,0.000000,0.000000,0.000000,8.608253,14.208843,18.197731,0.000000,2.418118,0.000000
2,2004-03-01,0.0,6.132983,2.768303,0.0,0.0,0.000000,0.000000,0.000000,2.093351,0.000000,0.000000,0.000000,9.016481,6.597724,16.537291,5.037995,8.828593,5.387750
3,2004-04-01,0.0,10.537711,0.000000,0.0,0.0,3.844978,3.598893,7.354399,0.000000,0.000000,0.000000,1.694580,9.532213,10.064083,11.832085,2.341915,0.000000,0.000000
4,2004-05-01,0.0,7.344062,0.000000,0.0,0.0,0.000000,2.376195,2.340261,0.000000,0.000000,1.500651,2.683400,8.077286,13.814450,11.136356,7.196212,6.196599,3.525939


In [5]:
source = ColumnDataSource({c: v.values for c, v in trends.items()})

# Select Widget
initial_value = "palau_travel"
col_lst = [col for col in trends.columns if col.endswith("travel")]

p = figure(plot_height=600, plot_width=880, x_axis_type="datetime")
line = p.line("date", initial_value, source=source, name=initial_value)

dropdown = Select(title="Country", value=initial_value,
                  options=col_lst, width=200)
dropdown.js_on_change("value", CustomJS(args=dict(line=line),
                                        code="line.glyph.y = {field: cb_obj.value};"))

show(column(dropdown, p))